# Reading Movies

In [10]:
import ast
import csv
final_dataset = []
with open('FinalDataset.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        final_dataset.append(item)

In [11]:
len(final_dataset)

2505

In [12]:
import os

with_errors = []
with open("application.log.bk") as f:
    for line in f.readlines():
        if "Processing" not in line:
            with_errors.append(line.split("Error in")[1].split(",")[0].strip())

    for file in with_errors:
        print(file)
#        try:
#            os.remove('./videos/'+file)
#        except Exception as e:
#            print(e)


1334
1343
1026
664
652


In [13]:
import queue

tasks_queue = queue.Queue()

downloaded = os.listdir('./videos')


missing_to_download = []
for record in final_dataset:
    if record["video_url"] != "":
        if record["result_uniqueID"] not in downloaded:
            missing_to_download.append(record["result_uniqueID"])
            tasks_queue.put(record)

print("Missing to download {}".format(missing_to_download))
print("Q Len {}".format(tasks_queue.qsize()))

Missing to download []
Q Len 0


In [5]:
pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [6]:
from threading import Thread
from pytube import YouTube
import logging

logging.basicConfig(filename='application.log', level=logging.INFO)

In [7]:
class VideoDownloaderWorker(Thread):
    def __init__(self, worker_number, tasks_queue):
        Thread.__init__(self)
        self.worker_number = worker_number
        self.tasks_queue = tasks_queue

    def run(self):
        while True:
            record = self.tasks_queue.get()

            if record == "END":
                break

            logging.info("Worker Number {}, Processing  {}".format(self.worker_number ,record["result_uniqueID"]))
            try:
                youtubeObject = YouTube(record["video_url"])
                youtubeObject = youtubeObject.streams.get_highest_resolution()
                youtubeObject.download(filename=record["result_uniqueID"], output_path="./videos/")
            except Exception as e:
                logging.error("Worker Number {}, Error in {}, {}".format(self.worker_number, record["result_uniqueID"], str(e)))
            self.tasks_queue.task_done()


In [8]:
workers_size = 30

for worker_number in range(workers_size):
    tasks_queue.put("END")

workers = []

for worker_number in range(workers_size):
    worker = VideoDownloaderWorker(worker_number, tasks_queue)
    worker.start()
    workers.append(worker)

for worker in workers:
    worker.join()

In [3]:
import os
path="./videos/"
import shutil
for filename in os.listdir(path):
    #print(filename)
    shutil.copyfile("./videos/{}".format(filename), "./renamed_videos/{}.mp4".format(filename))
    #os.rename(
    #break